In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Read in the data
demand_points = gpd.read_file('data/derived_data/kmeans_clusters.gpkg')
potential_sites = gpd.read_file('data/derived_data/all_pot_sites.gpkg')
matrix=pd.read_csv('data/derived_data/distance_matrix_walking.csv')
rcps=gpd.read_file('data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')

ERROR 1: PROJ: proj_create_from_database: Open of /home/silas/miniconda3/envs/geoenv/share/proj failed


In [2]:
matrix

,ID,cluster_ID,Walking_Duration_Minutes
0,e_1,0,51.808333
1,e_1,1,13.916667
2,e_1,2,72.901667
3,e_1,3,43.443333
4,e_1,4,64.725000
...,...,...,...
842395,p_9172,1195,91.175000
842396,p_9172,1196,55.048333
842397,p_9172,1197,137.096667
842398,p_9172,1198,74.121667


,cluster_id,kmeans_cluster,total_est_pop,geometry
0,0,0,471.192462,POINT (8.52515 47.39562)
1,1,1,400.295609,POINT (8.56716 47.36303)
2,2,2,420.194249,POINT (8.48960 47.37655)
3,3,3,523.218863,POINT (8.52709 47.35640)
4,4,4,450.826562,POINT (8.56113 47.41215)
...,...,...,...,...
1195,1195,1195,307.666667,POINT (8.58954 47.40300)
1196,1196,1196,312.381570,POINT (8.55215 47.40576)
1197,1197,1197,442.644664,POINT (8.52098 47.33991)
1198,1198,1198,200.000000,POINT (8.50408 47.38084)


In [9]:
distance_matrix = matrix.pivot(index='Object_ID', columns='Demand_Point_ID', values='Walking_Duration')

# get global mean and std of matrix
global_mean = distance_matrix.mean().mean()
global_std = distance_matrix.stack().std()

# generate synthetic data wiht same mean and std
size = [1200, 645]
synthetic_data = np.random.normal(global_mean, global_std, size=size)


# Get the lowest n entries for each column in synthetic_data
n = 50
for j in range(synthetic_data.shape[1]):
    column = synthetic_data[:, j]
    threshold = np.partition(column, n-1)[n-1]  # Get 15th smallest value
    mask = column > threshold
    synthetic_data[mask, j] = 10000000  # Set all values above threshold to penalty

# print number of entries above threshold
print((synthetic_data != 10000000).sum())

KeyError: 'Object_ID'

In [8]:
distance_matrix = matrix.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')


# Get the lowest n entries for each column in distance_matrix
n = 25
for column in distance_matrix.columns:
    # Convert column to numpy array for efficient computation
    values = distance_matrix[column].values
    threshold = np.partition(values, n-1)[n-1]  # Get nth smallest value
    mask = values > threshold
    distance_matrix.loc[mask, column] = 10000000  # Set all values above threshold to penalty

# print number of entries below penalty value
print((distance_matrix != 10000000).sum().sum())

30008


In [4]:
matrix

,ID,cluster_ID,Walking_Duration_Minutes
0,e_1,0,51.808333
1,e_1,1,13.916667
2,e_1,2,72.901667
3,e_1,3,43.443333
4,e_1,4,64.725000
...,...,...,...
842395,p_9172,1195,91.175000
842396,p_9172,1196,55.048333
842397,p_9172,1197,137.096667
842398,p_9172,1198,74.121667


In [37]:
import pulp

# Parameter: number of NEW facilities to open (in addition to already open sites)
p = 11

# Get dimensions from the real distance matrix
num_demand_points = distance_matrix.shape[1]  # number of demand points
num_potential_sites = distance_matrix.shape[0]  # number of potential sites
I = range(num_demand_points)
J = range(num_potential_sites)

# Identify already open sites and potential sites
existing = [j for j in J if potential_sites.iloc[j]['status'] == "open"]
new_sites = [j for j in J if potential_sites.iloc[j]['status'] == "potential"]

# Create the model
prob = pulp.LpProblem("P-Median_Problem", pulp.LpMinimize)

# Decision Variables: y only for potential new sites
y = pulp.LpVariable.dicts("Facility", new_sites, cat='Binary')

# Assignment decision variables (for all sites)
x = pulp.LpVariable.dicts("Assign",
                          [(i, j) for i in I for j in J],
                          cat='Binary')

# Objective Function: Minimize total distance
prob += pulp.lpSum(distance_matrix.iloc[j, i] * x[i, j] for i in I for j in J)

# Constraint: Each demand point is assigned to exactly one facility.
for i in I:
    prob += pulp.lpSum(x[i, j] for j in J) == 1

# Constraint: A demand point can only be assigned to an open facility.
for i in I:
    for j in J:
        if j in new_sites:
            # For potential sites, assignment allowed only if facility is selected (y[j]==1)
            prob += x[i, j] <= y[j]
        else:
            # Existing sites are already open
            prob += x[i, j] <= 1

# Constraint: Exactly p new facilities are opened (among potential sites).
prob += pulp.lpSum(y[j] for j in new_sites) == p

# Solve the model using Gurobi
prob.solve(pulp.GUROBI(msg=True))

# Results
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Distance: {pulp.value(prob.objective)}")

# Get new opened facilities from potential sites
opened_new = [j for j in new_sites if pulp.value(y[j]) == 1]
print(f"Number of new facilities opened: {len(opened_new)}")

# Combine existing with new opened sites for the full set of open facilities
opened_facilities = existing + opened_new
print(f"Total number of open facilities (existing + new): {len(opened_facilities)}")

# Create a dataframe of selected sites
selected_sites = potential_sites.iloc[opened_facilities].copy()
print("\nSelected Sites:")
print(selected_sites)

# Assignments: create a dictionary of demand point assignments.
assignments = {(i, j): pulp.value(x[i, j]) for i in I for j in J if pulp.value(x[i, j]) == 1}
print("\nFirst 10 Demand Point Assignments:")
counter = 0
for (i, j), _ in assignments.items():
    print(f"Demand Point {i} is assigned to Facility {j}")
    counter += 1
    if counter == 10:
        break

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "EndeavourOS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 22 physical cores, 22 logical processors, using up to 22 threads

Academic license 2616816 - for non-commercial use only - registered to ss___@student.ethz.ch
Optimize a model with 843601 rows, 842926 columns and 2316526 nonzeros
Model fingerprint: 0x994c7f2e
Variable types: 0 continuous, 842926 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1.139000e+10
Presolve removed 739900 rows and 738700 columns
Presolve time: 1.09s
Presolved: 103701 rows, 104226 columns, 309226 nonzeros
Found heuristic solution: objective 5032.4516667
Variable types: 0 continuous, 104226 integer (104226 binary)
Performing another presolve...
Presolve removed 97639 rows and 98010 column

In [38]:
selected_sites.to_file('data/derived_data/selected_sites_optimisation.gpkg', driver='GPKG')

In [39]:
import folium


# Convert selected_sites to WGS 84 if not already in lat/lon
selected_sites_4326 = selected_sites.to_crs(epsg=4326)

# Set the map center to the mean coordinates of all sites
mean_lat = selected_sites_4326.geometry.y.mean()
mean_lon = selected_sites_4326.geometry.x.mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Add markers with different colors:
# - Open sites (existing) in blue
# - New sites (potential) in red
for idx, row in selected_sites_4326.iterrows():
    coords = [row.geometry.y, row.geometry.x]
    color = 'blue' if row['status'] == 'open' else 'red'
    folium.Marker(location=coords, popup=row['ID'], icon=folium.Icon(color=color)).add_to(m)

m



In [36]:
# create folium map with all potential sites
t = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Add markers with different colors:
pot_sites=potential_sites.query('status=="potential"')



for idx, row in pot_sites.to_crs(epsg=4326).iterrows():
    coords = [row.geometry.y, row.geometry.x]
    color = 'blue' if row['status'] == 'open' else 'red'
    folium.Marker(location=coords, popup=row['ID'], icon=folium.Icon(color=color)).add_to(t)

t